# Practice 2 : Working with Key-Value Pairs

1. [Counting Words](#1.-Counting-Words)
1. [Recap](#Recap)
1. [References](#References)

## 1. Counting Words

*This exercise material is taken from [edX - Scalable Machine Learning Lab2](https://github.com/spark-mooc/mooc-setup/blob/master/ML_lab2_word_count_student.ipynb).*

The volume of unstructured text in existence is growing dramatically, and Spark is an excellent tool for analyzing this type of data. In this lab, we will write code that calculates the most common words in the Complete Works of William Shakespeare retrieved from Project Gutenberg.

### 1.1 Capitalization and punctuation
Real world files are more complicated than the data we have been using sor far. Some of the issues we have to address are:

- Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
- All punctuation should be removed.
- Any leading or trailing spaces on a line should be removed.

Define the function `remove_punct` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces. Use the Python `re` module to remove any text that is not a letter, number, or space. Reading help(re.sub) might be useful.

In [1]:
import re
from string import punctuation
def remove_punct(word):
    return re.sub(r'[{}‘—’”“]'.format(punctuation), " ", word).strip()

print(remove_punct('Hi, you!'))
print(remove_punct(' No under_score!'))
print(remove_punct(' *      Remove punctuation then spaces  * '))
print(remove_punct(" The Elephant's (4 cats). "))

Hi  you
No under score
Remove punctuation then spaces
The Elephant s  4 cats


In [19]:
print("   1   111 ***     ".strip())

1   111 ***


In [20]:
re.sub('\s{1,}', ' ', "   1   111 ***     ")

' 1 111 *** '

### 1.2 Load from a text file

Create a new RDD from the text files in Shakespeare's comedies folder. The filename and the path are already configured in `filename`. 

Once the RDD is created, apply the `remove_punct` transformation and check the first 10 elements.

In [ ]:
import pyspark

In [16]:
help(pyspark.SparkConf)

Help on class SparkConf in module pyspark.conf:

class SparkConf(builtins.object)
 |  Configuration for a Spark application. Used to set various Spark
 |  parameters as key-value pairs.
 |  
 |  Most of the time, you would create a SparkConf object with
 |  C{SparkConf()}, which will load values from C{spark.*} Java system
 |  properties as well. In this case, any parameters you set directly on
 |  the C{SparkConf} object take priority over system properties.
 |  
 |  For unit tests, you can also call C{SparkConf(false)} to skip
 |  loading external settings and get the same configuration no matter
 |  what the system properties are.
 |  
 |  All setter methods in this class support chaining. For example,
 |  you can write C{conf.setMaster("local").setAppName("My app")}.
 |  
 |  .. note:: Once a SparkConf object is passed to Spark, it is cloned
 |      and can no longer be modified by the user.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, loadDefaults=True, _jvm=None, _jconf

In [3]:
sc = pyspark.SparkContext()

In [10]:
import os.path
basedir = '/project/datasets'
inputpath = os.path.join('shakespeare', 'comedies', '*')
filename = os.path.join(basedir, inputpath)

In [11]:
filename

'/project/datasets/shakespeare/comedies/*'

In [12]:
! ls /project/datasets/shakespeare/comedies/

allswellthatendswell  merchantofvenice	     tempest
asyoulikeit	      merrywivesofwindsor    troilusandcressida
comedyoferrors	      midsummersnightsdream  twelfthnight
cymbeline	      muchadoaboutnothing    twogentlemenofverona
loveslabourslost      periclesprinceoftyre   winterstale
measureforemeasure    tamingoftheshrew


In [13]:
shakespeareComedies = sc.textFile(filename).map(remove_punct)

In [14]:
shakespeareComedies.take(10)

['A MIDSUMMER NIGHT S DREAM',
 '',
 '',
 'DRAMATIS PERSONAE',
 '',
 '',
 'THESEUS\tDuke of Athens',
 '',
 'EGEUS\tfather to Hermia',
 '']

### 1.2 Words from lines 

Before we can count the words' frequency, we have to address two issues with the format of the RDD:

- The first issue is that that we need to split each line by its spaces.
- The second issue is we need to filter out empty lines.

Apply a transformation that will split each element of the RDD.

Words can be divided by other characters than simply space, for example tabs (`\t`). Make sure you cover every case when splitting the lines. Python function `str.split` only covers the case where we want to split the words using a single separating character. If we want multiple characters, we need to look at [`re.split`](https://docs.python.org/3/library/re.html#re.split).

In [25]:
comedies_word = shakespeareComedies.flatMap(lambda word: word.split())

In [30]:
''.split()

[]

### 1.3 Remove empty elements

The next step is to filter out the empty elements. Remove all entries where the word is ''.

In [37]:
comedies_word_only = comedies_word.filter(lambda word: len(word) > 0 )

In [31]:
'This'

'This'

In [32]:
bool('This')

True

In [33]:
bool('')

False

### 1.4 Count the words 

We now have an RDD that is only words. The next step is to transform this RDD in a key-value pair RDD and count the word. 

Once this is done, return the 10 least common word in the dataset.

In [38]:
comedies_word_only.take(10)

['A',
 'MIDSUMMER',
 'NIGHT',
 'S',
 'DREAM',
 'DRAMATIS',
 'PERSONAE',
 'THESEUS',
 'Duke',
 'of']

In [39]:
comedies_word_pair = comedies_word_only.map(lambda word: (word, 1))

In [41]:
comedies_word_pair.take(5)

[('A', 1), ('MIDSUMMER', 1), ('NIGHT', 1), ('S', 1), ('DREAM', 1)]

In [42]:
word_frequency = comedies_word_pair.reduceByKey(lambda a, b: a + b)

In [43]:
word_frequency.first()

('to', 7082)

In [46]:
local_word_frequency = word_frequency.collectAsMap()

In [50]:
local_word_frequency['book']

42

In [64]:
word_frequency.top(10)

[('zodiacs', 1),
 ('zephyrs', 1),
 ('zenith', 1),
 ('zealous', 2),
 ('zeal', 10),
 ('zany', 1),
 ('zanies', 1),
 ('youths', 2),
 ('youthful', 12),
 ('youth', 149)]

In [68]:
word_frequency.top(10, key=lambda pair: -pair[1])

[('Stir', 1),
 ('funerals', 1),
 ('prevailment', 1),
 ('shady', 1),
 ('prosecute', 1),
 ('collied', 1),
 ('transpose', 1),
 ('Skim', 1),
 ('bouncing', 1),
 ('hempen', 1)]

In [58]:
help(word_frequency.top)

Help on method top in module pyspark.rdd:

top(num, key=None) method of pyspark.rdd.PipelinedRDD instance
    Get the top N elements from an RDD.
    
    .. note:: This method should only be used if the resulting array is expected
        to be small, as all the data is loaded into the driver's memory.
    
    .. note:: It returns the list sorted in descending order.
    
    >>> sc.parallelize([10, 4, 2, 12, 3]).top(1)
    [12]
    >>> sc.parallelize([2, 3, 4, 5, 6], 2).top(2)
    [6, 5]
    >>> sc.parallelize([10, 4, 2, 12, 3]).top(3, key=str)
    [4, 3, 2]



In [65]:
('a', 100) > ('a', 2)

True

In [63]:
'Serenity' > 'Eleanor'

True

### 1.5 Halt the SparkContext

## 2. Recap

In this notebook, we used and learned about the following parts of 
**[Python Spark API](http://spark.apache.org/docs/latest/api/python/)**:
2. Create an RDD from text files:
**[`SparkContext.textFile(path)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.textFile)**
5. Apply a transformation on each element of an RDD:
**[`RDD.map(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map)**
5. Apply a transformation on each element of an RDD  then flatten the results.:
**[`RDD.flatMap(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.flatMap)**
5. Filter an RDD:
**[`RDD.filter(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.filter)**
6. Merge the values for each keys: 
**[`RDD.reduceByKey(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey)**
7. Get the N elements from a RDD ordered in ascending order: **[`RDD.takeOrdered(N)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered)**


## 3. References

* [O'Reilly Learning Spark - Holden Karau, Andy Konwinski, Patrick Wendell, and Matei Zaharia](http://shop.oreilly.com/product/0636920028512.do)
* [Heather Miller - Parallel Programming and Data Analysis](http://heather.miller.am/teaching/cs212/slides/)